In [14]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import count, sum,substring,col

spark = SparkSession \
    .builder \
    .config("spark.jars",
            "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
    .master("local[*]") \
    .appName("leetcode-problems") \
    .getOrCreate()

creators = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/namastesql") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "creators") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

posts = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/namastesql") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "posts") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [2]:
creators.show()

+----------+---------------+---------+
|creator_id|   creator_name|followers|
+----------+---------------+---------+
|         1|Pavitra Nataraj|      900|
|         2|   Ankit Bansal|   150000|
|         3|    Rahul Gupta|    70000|
|         4|  Shomil Sharma|    55000|
|         5|     Neha Verma|    10000|
|         6|    Arjun Mehta|     4500|
|         7|   Kavya Sharma|    25000|
|         8|   Rishi Kapoor|   300000|
|         9|  Aditi Kasture|   120000|
|        10|     Vikram Rao|      600|
|        11|  Namrata Joshi|    70000|
|        12|Sameer Malhotra|     9000|
|        13|    Sonia Dutta|    34000|
|        14|   Karan Thakur|    52000|
|        15|    Rahul Singh|    95000|
+----------+---------------+---------+



In [3]:
posts.show()

+-------+----------+------------+-----------+
|post_id|creator_id|publish_date|impressions|
+-------+----------+------------+-----------+
|     p1|         1|  2023-12-03|     120000|
|    p10|         4|  2023-12-04|      71000|
|    p11|         5|  2023-10-15|       8000|
|    p12|         6|  2023-12-21|        600|
|    p13|         7|  2023-12-19|       1500|
|    p14|         8|  2023-09-05|      95000|
|    p15|         8|  2023-12-07|      35000|
|    p16|         8|  2023-12-18|      42000|
|    p17|         9|  2023-12-25|      36000|
|    p18|         9|  2023-12-05|      45000|
|    p19|         9|  2023-12-14|      38000|
|     p2|         2|  2023-12-02|      51000|
|    p20|         9|  2023-12-28|      42000|
|    p21|        10|  2023-11-30|        200|
|    p22|        11|  2023-12-06|      30000|
|    p23|        11|  2023-12-15|      42000|
|    p24|        11|  2023-12-22|      35000|
|    p25|        12|  2023-12-10|        900|
|    p26|        13|  2023-12-03| 

In [16]:
creators\
    .join(posts,creators.creator_id==posts.creator_id,"inner")\
    .select(creators.creator_name,posts.post_id,posts.impressions,posts.publish_date)\
    .withColumn("yearmonth",substring(posts.publish_date,1,7))\
    .filter(col("yearmonth")=="2023-12")\
    .groupby(creators.creator_name)\
    .agg(count(posts.post_id).alias("total_posts"),sum(posts.impressions).alias("sum_impressions"))\
    .filter((col("total_posts")>=3) & (col("sum_impressions")>100000))\
    .show()

+-------------+-----------+---------------+
| creator_name|total_posts|sum_impressions|
+-------------+-----------+---------------+
|Aditi Kasture|          4|         161000|
|  Rahul Singh|          3|         119000|
| Ankit Bansal|          3|         132000|
|Namrata Joshi|          3|         107000|
+-------------+-----------+---------------+

